<a href="https://colab.research.google.com/github/lupis30puc/bipm_text_analytics_exercises/blob/main/COVID_similarity_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import en_core_web_sm
import csv
import pickle
from google.colab import files

from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
from nltk.corpus import stopwords

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from scipy.spatial.distance import cosine, cdist

In [ ]:
len(diabetes_df)

36

In [ ]:
diabetes_df= pickle.load(open('/content/diabetes_new_target.pkl 23.12.53', 'rb'))
diabetes_df.head()

,index,publish_time,title,abstract,cord_uid,doi,journal,url,pdf_json_files,date,body_text,target_or_not
0,5,2020-03-11,The 24-Form Tai Chi Improves Anxiety and Depre...,anxieti depss coronari heart chd pcutan corona...,bc6i5izr,10.3389/fphys.2020.00149,Front Physiol,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,document_parses/pdf_json/4989f4d799ed70346c882...,2020-03-11,Coronavirus disease 2019 (COVID -19) has been ...,second most similar
1,8,2020-04-22,Wirtschaft unter Schock — Finanzpolitik hält d...,accord lead gman econom institut gman economi ...,17xwt5tk,10.1007/s10273-020-2629-z,Wirtschaftsdienst,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,2020-04-22,SARS-CoV-2 (COVID-19) emerged in December 2019...,target
2,12,2020-05-06,Practical Considerations When Performing Neuro...,led wid short pson equip ppe incas stss sourc ...,72s8wpla,10.1080/21646821.2020.1756132,Neurodiagn J,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,document_parses/pdf_json/d049cc13cfc0854eef691...,2020-05-06,The COVID-19 pandemic continues to escalate. T...,target
3,18,2020-03-10,Oestrogen Receptor β Activation Protects Again...,purpos oestrogen ceptor believ ext cardio isch...,ldjtf14i,10.1007/s10557-020-06949-3,Cardiovasc Drugs Ther,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,document_parses/pdf_json/44eb4b44d924b819f1040...,2020-03-10,The outbreak and unprecedented spread of SARS-...,target
4,20,2020-05-15,Independent risk factors and the long-term out...,investig incid continu nal plac thapi crrt und...,4rne67ia,10.1186/s13019-020-01153-8,J Cardiothorac Surg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,document_parses/pdf_json/070b9aa48069a446c6423...,2020-05-15,Despite worldwide efforts to prevent the sprea...,target


## Loading Datasets

In [ ]:
# Loading the filtered original dataset
uploaded=files.upload()

Saving meta_stemmed (1) to meta_stemmed (1) (1)


In [ ]:
# saving the filtered original dataset into a dataframe
meta = pickle.load(open('meta_stemmed (1) (1)', 'rb'))
meta.head()

,level_0,publish_time,title,abstract,date,year,language
0,4131,2020-01-09,Cocaine and amphetamine-regulated transcript p...,iolog cocain amphetamin gul transcript ppropep...,2020-01-09,2020,en
1,4195,2020-01-21,Pioneering Experience of Uniportal Video-Assis...,optim wai tat thorac scoliosi controvsi conven...,2020-01-21,2020,en
2,4310,2020-02-10,Epidemiological and clinical characteristics o...,sult surveil iolog charactist car hai eldli in...,2020-02-10,2020,en
3,4358,2020-02-07,Therapeutic strategies in an outbreak scenario...,origin psent spiratori curnt effort foc contai...,2020-02-07,2020,en
4,4485,2020-01-17,Single-Virus Tracking: From Imaging Methodolog...,im text un mechan assembl crucial pvent spad t...,2020-01-17,2020,en


In [ ]:
# resenting the index to fix an issue...
meta.reset_index(drop=True, inplace=True)
meta.head()

,level_0,publish_time,title,abstract,date,year,language
0,4131,2020-01-09,Cocaine and amphetamine-regulated transcript p...,iolog cocain amphetamin gul transcript ppropep...,2020-01-09,2020,en
1,4195,2020-01-21,Pioneering Experience of Uniportal Video-Assis...,optim wai tat thorac scoliosi controvsi conven...,2020-01-21,2020,en
2,4310,2020-02-10,Epidemiological and clinical characteristics o...,sult surveil iolog charactist car hai eldli in...,2020-02-10,2020,en
3,4358,2020-02-07,Therapeutic strategies in an outbreak scenario...,origin psent spiratori curnt effort foc contai...,2020-02-07,2020,en
4,4485,2020-01-17,Single-Virus Tracking: From Imaging Methodolog...,im text un mechan assembl crucial pvent spad t...,2020-01-17,2020,en


In [ ]:
for i in range(len(smoke_.abstract)):
        smoke_.abstract[i] = strip_multiple_whitespaces(str(smoke_.abstract[i])) #Remove multiple whitespaces
        smoke_.abstract[i] = strip_short(str(smoke_.abstract[i]), minsize=3) #Remove short words
        smoke_.abstract[i] = strip_numeric(str(smoke_.abstract[i])) #Remove digits
        smoke_.abstract[i] = strip_punctuation(str(smoke_.abstract[i]))  #Remove punctuation

In [ ]:
# Making the stemmed abstracts into a list
meta_abstracts = list(meta.abstract)
meta_abstracts[0]

'iolog cocain amphetamin gul transcript ppropeptid cartpt gene polymorph modifi diet obes lationship intact cartpt gene polymorph diet qualiti indic investig curnt intact major dietari indic diet qualiti intn dqi eat hei cartpt gene variant appar obes iranian cross section carri emploi appar obes diet qualiti diet qualiti intn dqi eat hei item semi quantit valid food fquenc questionnair cartpt gene polymorph genotyp polymas chain action strict frment length polymorph pcr–rflp techniqu concentr glycem mark lipid profil melanocyt stimul hormon msh outi lat peptid rp ana intact gene diet intact sult intact cartpt gene polymorph hei affect bmr intact sum glucos intact densiti lipoprotein cholestol hdl concentr intact adjt gene diet intact cartpt genotyp dqi tm fat mass intact waist circumf intact bodi mass intact basal bol bmr intact sum fast glucos intact rp intact individu adjt confound curnt intact cartpt genotyp adh hei dqi sco obes lat anthropometr bol'

In [ ]:
# Loading the target tables dataset
uploaded=files.upload()

Saving smoking_stemmed to smoking_stemmed


In [ ]:
target_df= pickle.load(open('smoking_stemmed', 'rb'))
target_df.head()

,Date,title,abstract
0,2020-05-22,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
1,2020-05-19,Factors Associated With Intubation and Prolong...,extub design trospect obsv set chico metropoli...
2,2020-05-19,Risk factors for adverse clinical outcomes in ...,hetogen vari prognosi charactist cours off gui...
3,2020-05-16,Severe obesity is associated with higher in-ho...,york curnt epicent undps minor pval obes appea...
4,2020-05-15,"Clinical Characteristics, Risk Factors and Out...",kuwait prior port ri mass scen incom travel ri...


In [ ]:
for i in range(len(target_df.abstract)):
        target_df.abstract[i] = strip_multiple_whitespaces(str(target_df.abstract[i])) #Remove multiple whitespaces
        target_df.abstract[i] = strip_short(str(target_df.abstract[i]), minsize=3) #Remove short words
        target_df.abstract[i] = strip_numeric(str(target_df.abstract[i])) #Remove digits
        target_df.abstract[i] = strip_punctuation(str(target_df.abstract[i]))  #Remove punctuation

In [ ]:
# Making the steemed abstracts into a list
smoke_target = list(target_df.abstract)
smoke_target[0]

'port ga design surveil network set ca race ethnic undli condit sult nineti pcent undli condit quir intens ca invas mechan ventil vasopssor di vsu adjt arr spect arr obes arr immunosuppss arr diabet arr vsu arr spect arr immunosuppss arr nal arr lung arr cardiovascular arr neurolog disord arr diabet arr race ethnic limit limit di complet chart abstract access cord exclud incas markedli incas charact pson est defin target pvent tatment stgi'

In [ ]:
print('We have this ' + str(len(smoke_target)) + ' many abstracts from the target table')
print('We have this ' + str(len(meta_abstracts)) + ' many abstracts from the filtered orginal dataset')

We have this 31 many abstracts from the target table
We have this 23234 many abstracts from the filtered orginal dataset


In [ ]:
# splitting filtered data corpus
corpus_data= [doc.split() for doc in meta_abstracts]
print(corpus_data[0])

['iolog', 'cocain', 'amphetamin', 'gul', 'transcript', 'ppropeptid', 'cartpt', 'gene', 'polymorph', 'modifi', 'diet', 'obes', 'lationship', 'intact', 'cartpt', 'gene', 'polymorph', 'diet', 'qualiti', 'indic', 'investig', 'curnt', 'intact', 'major', 'dietari', 'indic', 'diet', 'qualiti', 'intn', 'dqi', 'eat', 'hei', 'cartpt', 'gene', 'variant', 'appar', 'obes', 'iranian', 'cross', 'section', 'carri', 'emploi', 'appar', 'obes', 'diet', 'qualiti', 'diet', 'qualiti', 'intn', 'dqi', 'eat', 'hei', 'item', 'semi', 'quantit', 'valid', 'food', 'fquenc', 'questionnair', 'cartpt', 'gene', 'polymorph', 'genotyp', 'polymas', 'chain', 'action', 'strict', 'frment', 'length', 'polymorph', 'pcr–rflp', 'techniqu', 'concentr', 'glycem', 'mark', 'lipid', 'profil', 'melanocyt', 'stimul', 'hormon', 'msh', 'outi', 'lat', 'peptid', 'rp', 'ana', 'intact', 'gene', 'diet', 'intact', 'sult', 'intact', 'cartpt', 'gene', 'polymorph', 'hei', 'affect', 'bmr', 'intact', 'sum', 'glucos', 'intact', 'densiti', 'lipoprote

In [ ]:
# splitting target tables corpus
corpus_target= [doc.split() for doc in smoke_target]
print(corpus_target[0])

['port', 'ga', 'design', 'surveil', 'network', 'set', 'ca', 'race', 'ethnic', 'undli', 'condit', 'sult', 'nineti', 'pcent', 'undli', 'condit', 'quir', 'intens', 'ca', 'invas', 'mechan', 'ventil', 'vasopssor', 'di', 'vsu', 'adjt', 'arr', 'spect', 'arr', 'obes', 'arr', 'immunosuppss', 'arr', 'diabet', 'arr', 'vsu', 'arr', 'spect', 'arr', 'immunosuppss', 'arr', 'nal', 'arr', 'lung', 'arr', 'cardiovascular', 'arr', 'neurolog', 'disord', 'arr', 'diabet', 'arr', 'race', 'ethnic', 'limit', 'limit', 'di', 'complet', 'chart', 'abstract', 'access', 'cord', 'exclud', 'incas', 'markedli', 'incas', 'charact', 'pson', 'est', 'defin', 'target', 'pvent', 'tatment', 'stgi']


In [ ]:
# saving the corpus in pickle files ?
pickle.dump(corpus_data, open("corpus_data.pkl", "wb"))
pickle.dump(corpus_target, open("corpus_target.pkl", "wb"))

## *Using doc2vec:*

In [ ]:
# initiaize the model building vocabulary with the abstracts from the filtered original dataset
# we used more epochs because it improves 
data_sentences = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus_data)]
d2v_model = Doc2Vec(vector_size=20, min_count=5, workers=11, alpha=0.025, epochs=200)
d2v_model.build_vocab(data_sentences)
d2v_model.train(data_sentences, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)
d2v_model.random.seed(0)

In [ ]:
# Saving the model trainee with the orginal dataset
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")
d2v_model.save(fname)
model = Doc2Vec.load(fname) 
#model.random.seed(0)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Applying the model to the target datset
d2v_target = []
for i in range(len(corpus_target)):
  model.random.seed(0)
  d2v_target.append(model.infer_vector(corpus_target[i], epochs=200))

d2v_target[1]

array([-1.879537  ,  4.762292  , -5.85559   , -5.156556  , -1.1594985 ,
       -1.2757808 , -0.92790306,  4.07986   , -0.93774617,  0.78415465,
        3.1842926 , -1.4641929 ,  0.37450585,  4.876931  , -1.9438379 ,
       -1.3029863 , -0.9864957 ,  1.1568055 , -3.3962913 ,  0.94001704],
      dtype=float32)

In [ ]:
# saving the corpus in pickle files ?
pickle.dump(model, open("d2v_model_saved.pkl", "wb"))
pickle.dump(d2v_target, open("d2v_target.pkl", "wb"))

In [ ]:
# Checking if traget 1 is in the filtered dataset
check = smoke_.title.str.contains(target_df.title[1])

print(smoke_[check])

       level_0 publish_time  ...  year language
11125    62688         2020  ...  2020       en

[1 rows x 7 columns]


In [ ]:
#Cheking the embedding for the model equivalent to the target 1 
model[11190]

array([ 3.7311559 , -2.8480105 , -2.1895685 ,  3.2080095 , -1.1302752 ,
       -0.41551363, -1.7532947 , -1.6751802 , -2.6261349 ,  2.3104455 ,
        0.6694338 ,  0.99661225, -2.0332503 ,  5.168265  ,  2.1761572 ,
       -1.1135569 , -0.23366049, -0.9266206 ,  0.67136353, -0.7923474 ],
      dtype=float32)

In [ ]:
similars = model.docvecs.most_similar(positive=[d2v_target[1]], topn=3)
print(similars)

[(10970, 0.9892420768737793), (7054, 0.7739754319190979), (592, 0.7703381776809692)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print(meta['level_0'][10970])
print(meta['title'][10970])
print(meta['abstract'][10970])
print(smoke_target[1])

62688
Factors Associated With Intubation and Prolonged Intubation in Hospitalized Patients With COVID-19
extub design trospect obsv set chico metropolitan aa subject april sociodemograph charactist prolong spiratori failur secondari sult intquartil rang bodi mass intquartil rang eventu extub di mean fol diabet gss spiratori oxygen satur histori diabet short bath pdict bodi mass extub addit sign spiratori distss diabet quir obes prolong otolaryngo consult airwai man consid decis make
extub design trospect obsv set chico metropolitan aa subject april sociodemograph charactist prolong spiratori failur secondari sult intquartil rang bodi mass intquartil rang eventu extub di mean fol diabet gss spiratori oxygen satur histori diabet short bath pdict bodi mass extub addit sign spiratori distss diabet quir obes prolong otolaryngo consult airwai man consid decis make


## Confirming that all target articles are in the original database by checking the titles

In [ ]:
# Checking all titles of target table, doing a list of the found ones and the not found ones:
titles_not_found = []
index_titles_found = []
for i in range(len(target_df.title)):
  check = smoke_.title.str.contains(target_df.title[i])
  if len(smoke_[check]) == 0:
    titles_not_found.append([i, target_df.title[i]])
  else:
    i2 = int(list(smoke_[check].index)[0])
    index_titles_found.append([i, i2])

print(len(titles_not_found))
print(titles_not_found)
print(len(index_titles_found))
print(index_titles_found)

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


3
[[0, 'Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019 (COVID-19)-Associated Hospitalization Surveillance Network (COVID-NET)'], [12, 'Analysis of hospitalized COVID-19 patients in the Mount Sinai Health System using electronic medical records (EMR) reveals important prognostic factors for improved clinical outcomes'], [25, 'Risk factors for severe corona virus disease 2019 (COVID-19) patients : a systematic review and meta analysis']]
39
[[1, 11190], [2, 5801], [3, 20172], [4, 5417], [5, 9544], [6, 1071], [7, 6891], [8, 4918], [9, 4323], [10, 4323], [11, 7018], [13, 16353], [14, 16353], [15, 4626], [16, 13809], [17, 13809], [18, 24279], [19, 16009], [20, 16009], [21, 10500], [22, 10500], [23, 9516], [24, 7005], [26, 5339], [27, 9214], [28, 8479], [29, 8479], [30, 8479], [31, 7079], [32, 3294], [33, 3294], [34, 8047], [35, 7676], [36, 17515], [37, 17515], [38, 6151], [39, 15373], [40, 15373

In [ ]:
titles_not_found

[[0,
  'Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019 (COVID-19)-Associated Hospitalization Surveillance Network (COVID-NET)'],
 [12,
  'Analysis of hospitalized COVID-19 patients in the Mount Sinai Health System using electronic medical records (EMR) reveals important prognostic factors for improved clinical outcomes'],
 [25,
  'Risk factors for severe corona virus disease 2019 (COVID-19) patients : a systematic review and meta analysis']]

In [ ]:
# Checking the titles not found with just a part of the whole string.
check = smoke_.title.str.contains('Analysis of hospitalized COVID-19 patients in the Mount Sinai Health System using electronic medical records')
print(smoke_[check])

      level_0 publish_time  ...  year language
8530    51644   2020-05-04  ...  2020       en

[1 rows x 7 columns]


In [ ]:
# checking the target 12 that wasn't found by the complete title but it is found with just a part of the title...
# although the abstracts seem to be a bit different...
print(smoke_['title'][8530])
print(smoke_data[8530])
print(smoke_target[12])

Analysis of hospitalized COVID-19 patients in the Mount Sinai Health System using electronic medical records (EMR) reveals important prognostic factors for improved clinical outcomes
threat wid urgent undstand charactist have mark mortal featur design set particip obsv extract electron mount sinai exposur dinosi pcr assai nasal swab main race mortal demograph site behavior vital sign lab comorbid discharg death estim hispan african amican disproportion rel diff mortal obsv diff grai reflect diff lin undli comorbid factor mortal oxygen satur icu area elev creatinin alanin aminotransfas alt bodi mass bmi asthma length stai mortal deceas elev mark inflamm lin bmi oxygen satur white blood wbc creatinin alt prognost indic mortal relev race racial dispar inpati mortal singl ttiari race featur reduc mortal discharg grea surviv
threat wid urgent undstand charactist have mark mortal featur design set particip obsv extract electron mount sinai exposur dinosi pcr assai nasal swab main race mortal

In [ ]:
# checking the target 0 that wasn't found by the complete title but it is found with just a part of the title...
# although the abstracts seem to be a bit different...
print(smoke_['title'][6457])
print(smoke_data[6457])
print(smoke_target[0])

Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019 (COVID-19)-Associated Hospitalization Surveillance Network (COVID-NET)
state grea numb death factor sev adult design surveil network set state march sex race ethnic undli condit nineti pcent undli condit requir intens icu admiss invas mechan ventil vasopressor factor icu admiss vsu adjt arr respect male sex arr obes arr immunosuppress arr diabet arr factor mortal vsu arr respect male sex arr immunosuppress arr renal arr chronic lung arr cardiovascular arr neurolog disord arr diabet arr race ethnic icu admiss death limit limit discharg complet chart abstract access exclud mortal markedli charact pson est sev defin target stgi
state grea death factor adult design surveil network set state march sex race ethnic undli condit nineti pcent undli condit requir intens icu admiss invas mechan ventil vasopressor factor icu admiss vsu arr respect male sex

In [ ]:
# modify the titles so that they don't include the "()"
titles_not_found[0][1] = 'Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019'
titles_not_found[1][1] = 'Analysis of hospitalized COVID-19 patients in the Mount Sinai Health System using electronic medical records'
titles_not_found[2][1] = 'Risk factors for severe corona virus disease 2019'
print(titles_not_found)

[[0, 'Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019'], [12, 'Analysis of hospitalized COVID-19 patients in the Mount Sinai Health System using electronic medical records'], [25, 'Risk factors for severe corona virus disease 2019']]


In [ ]:
# appending to the index_title_found the articles which weren't found on the first round

not_found_again = []
for i in range(len(titles_not_found)):
   target_t = titles_not_found[i][1]
   check = smoke_.title.str.contains(target_t)
   if len(smoke_[check]) == 0:
     not_found_again.append([i, target_df.title[i]])
   else:
     i2 = int(list(smoke_[check].index)[0])
     
     index_titles_found.append([i, i2])

print(len(not_found_again))
print(not_found_again)
print(len(index_titles_found))
print(sorted(index_titles_found))

0
[]
42
[[0, 6457], [1, 8530], [1, 11190], [2, 5801], [2, 9419], [3, 20172], [4, 5417], [5, 9544], [6, 1071], [7, 6891], [8, 4918], [9, 4323], [10, 4323], [11, 7018], [13, 16353], [14, 16353], [15, 4626], [16, 13809], [17, 13809], [18, 24279], [19, 16009], [20, 16009], [21, 10500], [22, 10500], [23, 9516], [24, 7005], [26, 5339], [27, 9214], [28, 8479], [29, 8479], [30, 8479], [31, 7079], [32, 3294], [33, 3294], [34, 8047], [35, 7676], [36, 17515], [37, 17515], [38, 6151], [39, 15373], [40, 15373], [41, 6271]]


In [ ]:
# saving the index_titles_found in a sorted way
index_titles_found = sorted(index_titles_found)
print(index_titles_found)

[[0, 6457], [1, 8530], [1, 11190], [2, 5801], [2, 9419], [3, 20172], [4, 5417], [5, 9544], [6, 1071], [7, 6891], [8, 4918], [9, 4323], [10, 4323], [11, 7018], [13, 16353], [14, 16353], [15, 4626], [16, 13809], [17, 13809], [18, 24279], [19, 16009], [20, 16009], [21, 10500], [22, 10500], [23, 9516], [24, 7005], [26, 5339], [27, 9214], [28, 8479], [29, 8479], [30, 8479], [31, 7079], [32, 3294], [33, 3294], [34, 8047], [35, 7676], [36, 17515], [37, 17515], [38, 6151], [39, 15373], [40, 15373], [41, 6271]]


### Continue with titles found

In [ ]:
# Run the similarity test for all found articles: 
# WITH TITLES FOUND
similar_passed = []
similar_failed = []
for i in range(len(index_titles_found)):
   target_i = index_titles_found[i][0]
   sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[target_i]], topn=2)
   if sim_test[0][0] == index_titles_found[i][1]:
     similar_passed.append([index_titles_found[i][0], sim_test[0][0], sim_test[0][1], sim_test[1][0], sim_test[1][1]])
   else:
     similar_failed.append([index_titles_found[i], sim_test])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
# Check the articles that failed the similarity test
print(str(len(similar_failed)) + ' articles failed the similarity test. Because there are duplicates?') 
print(similar_failed)
for i in range(len(similar_failed)):
  try:
    # get the index for the target table
    t = similar_failed[i][0][0]
    print(t)
    print(target_df.title[t])
    # get the first most similar article
    f = similar_failed[i][1][0][0]
    print(f)
    print(smoke_.title[f])
    # get the second most similar article
    s = similar_failed[i][1][1][0]
    print(s)
    print(smoke_.title[s])
  except:
    KeyError

11 articles failed the similarity test. Because there are duplicates?
[[[1, 8530], [(11190, 0.9971437454223633), (22637, 0.8147382736206055)]], [[2, 9419], [(5801, 0.9954943060874939), (4226, 0.8569265007972717)]], [[6, 1071], [(11614, 0.9964664578437805), (1071, 0.9916489124298096)]], [[14, 16353], [(22891, 0.9955148696899414), (16353, 0.9823817014694214)]], [[17, 13809], [(24636, 0.9954515099525452), (13809, 0.962853193283081)]], [[21, 10500], [(19769, 0.9936889410018921), (10500, 0.9886385202407837)]], [[22, 10500], [(19769, 0.9936889410018921), (10500, 0.9886385202407837)]], [[29, 8479], [(16925, 0.9896647334098816), (24017, 0.9891713857650757)]], [[30, 8479], [(16925, 0.9896647334098816), (24017, 0.9891713857650757)]], [[32, 3294], [(16987, 0.9955422878265381), (6410, 0.993366003036499)]], [[33, 3294], [(16987, 0.9934666156768799), (3294, 0.9900428652763367)]]]
1
Factors Associated With Intubation and Prolonged Intubation in Hospitalized Patients With COVID-19
11190
Factors Associ

In [ ]:
# Check the articles that passed the similarity test
print(str(len(similar_passed)) + ' articles passed the similarity test') 
similar_passed[:3]

31 articles passed the similarity test


[[0, 6457, 0.9959843158721924, 6369, 0.8455496430397034],
 [1, 11190, 0.9971437454223633, 22637, 0.8147382736206055],
 [2, 5801, 0.9954943060874939, 4226, 0.8569265007972717]]

In [ ]:
# IT SEEMS THAT THE DUPLICATES IN THE TARGET TABLE WEREN'T REMOVED 
# AS WELL AS THE DUPLICATES IN THE ORIGINAL DATABASE

In [ ]:
type(target_df.title)

pandas.core.series.Series

In [ ]:

""" 
# Run the similarity test for all found titles:

target_similar = []
for i in range(len(index_titles_found)):
   target_i = index_titles_found[i][0]
   sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[target_i]], topn=3)
   if sim_test[0][0] == index_titles_found[i][1]:
     # preparing the list to include all info
     target_similar.append([target_i, target_df.title[target_i], smoke_target[target_i], 
                            sim_test[0][0], sim_test[0][1], smoke_.title[sim_test[0][0]], smoke_data[sim_test[0][0]],
                            sim_test[1][0], sim_test[1][1], smoke_.title[sim_test[1][0]], smoke_data[sim_test[1][0]]])
"""


## Doing a function to get a dataframe with top similar docs

In [ ]:
target_df.columns

Index(['Date', 'title', 'abstract'], dtype='object')

In [ ]:
meta.columns

Index(['level_0', 'publish_time', 'title', 'abstract', 'date', 'year',
       'language'],
      dtype='object')

In [ ]:
# Making a function for getting the top similar to add to the target table

def get_similar_docs(target_df, meta_df)

# Run the similarity test assuming all titles are in the filtered dataset:
  similar_to_target = []
  for i in range(len(target_df.title)):
    sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[i]], topn=3)
    #this way the list could be used to create a dataframe
    similar_to_target.append([i, 'target', 1, target_df.title[i], target_df.abstract[i]])
    similar_to_target.append([sim_test[0][0], 'most similar', sim_test[0][1], meta_df.title[sim_test[0][0]], meta_df.abstract[sim_test[0][0]]])
    similar_to_target.append([sim_test[1][0], 'second most similar', sim_test[1][1], meta_df.title[sim_test[1][0]], meta_df.abstract[sim_test[1][0]]])                        
    similar_to_target.append([sim_test[2][0], 'third most similar', sim_test[2][1], meta_df.title[sim_test[0][0]], meta_df.abstract[sim_test[0][0]]])

    # creating a dataframe with the top 3 most similar docs of the target ones!
    df_colum = ['original_index', 'original_db', 'similarity_percentage', 'title', 'abstract']
    similar_to_target_df = pd.DataFrame(similar_to_target, index=range(len(similar_to_target)), columns=df_colum)
    new_docs_target_df = similar_to_target_df.drop_duplicates(subset='title', keep="first", inplace=False)
  
  print(new_docs_target_df.original_db.value_counts())
  return new_docs_target_df

In [ ]:
# Run the similarity test assuming all titles are in the filtered dataset:

similar_to_target = []
for i in range(len(target_df.title)):
   sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[i]], topn=3)
   #this way the list could be used to create a dataframe
   similar_to_target.append([i, 'target', 1, target_df.title[i], target_df.abstract[i]])
   similar_to_target.append([sim_test[0][0], 'most similar', sim_test[0][1], meta.title[sim_test[0][0]], meta.abstract[sim_test[0][0]]])
   similar_to_target.append([sim_test[1][0], 'second most similar', sim_test[1][1], meta.title[sim_test[1][0]], meta.abstract[sim_test[1][0]]])                        
   similar_to_target.append([sim_test[2][0], 'third most similar', sim_test[2][1], meta.title[sim_test[0][0]], meta.abstract[sim_test[0][0]]])
   #similar_to_target.append([i, target_df.title[i],
                           # sim_test[0][0], sim_test[0][1], smoke_.title[sim_test[0][0]], smoke_data[sim_test[0][0]],
                           # sim_test[1][0], sim_test[1][1], smoke_.title[sim_test[1][0]], smoke_data[sim_test[1][0]]],
                           # sim_test[2][0], sim_test[2][1], smoke_.title[sim_test[0][0]], smoke_data[sim_test[0][0]],)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
len(target_df)

31

In [ ]:
similar_to_target[0]

[0,
 'target',
 1,
 'Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019 (COVID-19)-Associated Hospitalization Surveillance Network (COVID-NET)',
 'port ga design surveil network set ca race ethnic undli condit sult nineti pcent undli condit quir intens ca invas mechan ventil vasopssor di vsu adjt arr spect arr obes arr immunosuppss arr diabet arr vsu arr spect arr immunosuppss arr nal arr lung arr cardiovascular arr neurolog disord arr diabet arr race ethnic limit limit di complet chart abstract access cord exclud incas markedli incas charact pson est defin target pvent tatment stgi']

In [ ]:
"""
df_colum = ['Target_index', 'Target_title', 
            'Most_sim_index', 'Most_sim_percnt', 'Most_sim_title', 'Most_sim_abstract', 
            '2_most_sim_index', '2_most_sim_percnt','2_most_sim_title', '2_most_sim_abstract',
            '3_most_sim_index', '3_most_sim_percnt','3_most_sim_title', '3_most_sim_abstract']
target_similar_df = pd.DataFrame(target_similar, index=range(len(target_similar)), columns=df_colum)
target_similar_df.head()
"""

In [ ]:
# creating a dataframe with the top 3 most similar docs of the target ones!
df_colum = ['original_index', 'original_db', 'similarity_percentage', 'title', 'abstract']
similar_to_target_df = pd.DataFrame(similar_to_target, index=range(len(similar_to_target)), columns=df_colum)
similar_to_target_df.head(12)

,original_index,original_db,similarity_percentage,title,abstract
0,0,target,1.000000,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
1,6424,most similar,0.990724,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
2,6336,second most similar,0.874410,Characteristics and predictors of hospitalizat...,knowledg individu ord target stgi gen appropri...
3,8966,third most similar,0.874329,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
4,1,target,1.000000,Factors Associated With Intubation and Prolong...,extub design trospect obsv set chico metropoli...
5,10970,most similar,0.989242,Factors Associated With Intubation and Prolong...,extub design trospect obsv set chico metropoli...
6,7054,second most similar,0.773975,"Delirium Incidence, Duration and Severity in C...",delirium incid durat intens ca known obsv larg...
7,592,third most similar,0.770338,Factors Associated With Intubation and Prolong...,extub design trospect obsv set chico metropoli...
8,2,target,1.000000,Risk factors for adverse clinical outcomes in ...,hetogen vari prognosi charactist cours off gui...
9,5770,most similar,0.995916,Risk factors for adverse clinical outcomes in ...,hetogen vari prognosi charactist cours off gui...


In [ ]:
similar_to_target_df.original_db.value_counts()

target                 31
second most similar    31
third most similar     31
most similar           31
Name: original_db, dtype: int64

In [ ]:
#new_docs_target_df = similar_to_target_df.copy()
#new_docs_target_df.head()

new_docs_target_df = similar_to_target_df.drop_duplicates(subset='title', keep="first", inplace=False)
new_docs_target_df.head()

,original_index,original_db,similarity_percentage,title,abstract
0,0,target,1.000000,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
2,6336,second most similar,0.874410,Characteristics and predictors of hospitalizat...,knowledg individu ord target stgi gen appropri...
4,1,target,1.000000,Factors Associated With Intubation and Prolong...,extub design trospect obsv set chico metropoli...
6,7054,second most similar,0.773975,"Delirium Incidence, Duration and Severity in C...",delirium incid durat intens ca known obsv larg...
8,2,target,1.000000,Risk factors for adverse clinical outcomes in ...,hetogen vari prognosi charactist cours off gui...


In [ ]:
len(new_docs_target_df)

60

In [ ]:
new_docs_target_df.original_db.value_counts()

target                 30
second most similar    29
most similar            1
Name: original_db, dtype: int64

In [ ]:
print(list(new_docs_target_df.original_db.value_counts()))

[30, 29, 1]


In [ ]:
new_docs_target_df[new_docs_target_df['original_db'] !='target']

,original_index,original_db,similarity_percentage,title,abstract
1,6424,most similar,0.990724,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
2,6336,second most similar,0.874410,Characteristics and predictors of hospitalizat...,knowledg individu ord target stgi gen appropri...
3,8966,third most similar,0.874329,Interim Analysis of Risk Factors for Severe Ou...,port ga design surveil network set ca race eth...
5,10970,most similar,0.989242,Factors Associated With Intubation and Prolong...,extub design trospect obsv set chico metropoli...
6,7054,second most similar,0.773975,"Delirium Incidence, Duration and Severity in C...",delirium incid durat intens ca known obsv larg...
...,...,...,...,...,...
118,6238,second most similar,0.815244,Clinical characteristics of 2019 novel coronav...,decemb spiratori ard emg citi rapidli spad sou...
119,2360,third most similar,0.796649,Clinical Characteristics of Coronavirus Diseas...,decemb emg citi rapidli spad charactist affect...
121,6238,most similar,0.993195,Clinical characteristics of 2019 novel coronav...,decemb spiratori ard emg citi rapidli spad sou...
122,8460,second most similar,0.835257,Transmission and clinical characteristics of c...,emigr escal spad citi port foc outsid characti...


In [ ]:
not_target = new_docs_target_df[new_docs_target_df['original_db'] !='target']
for i in not_target.original_db:
  print(i)

In [ ]:
# Removing duplicates
not_target = new_docs_target_df[new_docs_target_df['original_db'] !='target']
not_target.drop_duplicates(subset='title', keep="first", inplace=True)
print(not_target.head())
print(len(not_target))

   original_index  ...                                           abstract
1            6424  ...  port ga design surveil network set ca race eth...
2            6336  ...  knowledg individu ord target stgi gen appropri...
5           10970  ...  extub design trospect obsv set chico metropoli...
6            7054  ...  delirium incid durat intens ca known obsv larg...
9            5770  ...  hetogen vari prognosi charactist cours off gui...

[5 rows x 5 columns]
60


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# saving a list with the top 10 most similiar to target ones
target_top_similar = []
for i in range(len(target_df.title)):
   sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[i]], topn=10)
   target_top_similar.append({i:sim_test})ve

print(target_top_similar[0])
#pickle.dump(ingre_top_modes, open("ingredients_prep_modes_50000_2.pkl", "wb"))

{0: [(6457, 0.9936496615409851), (5265, 0.8906867504119873), (8833, 0.8109248876571655), (6810, 0.8076139688491821), (12865, 0.791982889175415), (6369, 0.7909993529319763), (14127, 0.7876187562942505), (8846, 0.7801713943481445), (1483, 0.7758084535598755), (9330, 0.7734847664833069)]}


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[0]], topn=10)
print(sim_test[0])

(6457, 0.9936496615409851)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
# check if the second most similar is on the target table:

for i in range(len(target_df.title)):
   sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[i], topn=5)
   second_sim = smoke_.title[sim_test[1][0]]
   test = target_df.title.str.contains(second_sim)
   if len(target_df[test]) != 0:
     print('target index:' + str(i))
     print('meta index:' + str(sim_test[1][0]))
     print(target_df[test])


#print(sim_test[0])




/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` i

target index:6
meta index:1071
         Date  ...                                           abstract
6  2020-05-12  ...  identif corona great aid deliv prop optim reso...

[1 rows x 3 columns]
target index:7
meta index:4626
          Date  ...                                           abstract
15  2020-04-29  ...  editori angiotensin convt enzym inhibitor acei...

[1 rows x 3 columns]
target index:9
meta index:19246
          Date  ...                                           abstract
9   2020-05-06  ...  itali european hit est death europ prospect co...
10  2020-05-06  ...  itali european hit est death europ prospect co...

[2 rows x 3 columns]
target index:10
meta index:19246
          Date  ...                                           abstract
9   2020-05-06  ...  itali european hit est death europ prospect co...
10  2020-05-06  ...  itali european hit est death europ prospect co...

[2 rows x 3 columns]
target index:13
meta index:22891
          Date  ...                         

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` i

          Date  ...                                           abstract
21  2020-04-01  ...  clarifi factor establish ian bett choos thapeu...
22  2020-04-01  ...  clarifi factor establish ian bett choos thapeu...

[2 rows x 3 columns]
target index:22
meta index:10500
          Date  ...                                           abstract
21  2020-04-01  ...  clarifi factor establish ian bett choos thapeu...
22  2020-04-01  ...  clarifi factor establish ian bett choos thapeu...

[2 rows x 3 columns]
target index:28
meta index:8479
          Date  ...                                           abstract
28  2020-03-27  ...  evolv dramat summaris mortal chronic obstruct ...
29  2020-03-27  ...  evolv dramat summar mortal chronic obstruct pu...
30  2020-03-27  ...  evolv dramat summar mortal chronic obstruct pu...

[3 rows x 3 columns]
target index:29
meta index:24017
          Date  ...                                           abstract
28  2020-03-27  ...  evolv dramat summaris mortal chron

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
# Checking the titles not found with just a part of the whole string.
check = smoke_.title.str.contains('Analysis of hospitalized COVID-19 patients in the Mount Sinai Health System using electronic medical records')
print(smoke_[check])

In [ ]:
checkt = target_df.title.str.contains(smoke_.title[6271
                                                   ])
if len(target_df[checkt]) != 0:
  print(target_df[checkt])

          Date  ...                                           abstract
41  2020-02-09  ...  decemb ard citi rapidli sought delin charactis...

[1 rows x 3 columns]


In [ ]:
# check if the second or/and third most similar is a duplicate for the target article:

### Trying out with finding the abstract with function str.contains

In [ ]:
check = smoke_.abstract.str.contains(target_df.abstract[0])
print(smoke_[check])

Empty DataFrame
Columns: [level_0, publish_time, title, abstract, date, year, language]
Index: []


In [ ]:
# Checking all abstracts of target table, doing a list of the found ones and the not found ones:
abstract_not_found = []
index_abstract_found = []
for i in range(len(target_df.abstract)):
  check = smoke_.abstract.str.contains(target_df.abstract[i])
  if len(smoke_[check]) == 0:
    abstract_not_found.append([i, target_df.abstract[i]])
  else:
    i2 = int(list(smoke_[check].index)[0])
    index_abstract_found.append([i, i2])

print(len(abstract_not_found))
print(abstract_not_found)
print(index_abstract_found)
print(len(index_abstract_found))

38
[[0, 'state grea death factor adult design surveil network set state march sex race ethnic undli condit nineti pcent undli condit requir intens icu admiss invas mechan ventil vasopressor factor icu admiss vsu arr respect male sex arr obes arr immunosuppress arr diabet arr factor mortal vsu arr respect male sex arr immunosuppress arr renal arr chronic lung arr cardiovascular arr neurolog disord arr diabet arr race ethnic icu admiss death limit limit discharg complet chart abstract access exclud mortal markedli charact pson est defin target stgi'], [2, 'hetogen vari prognosi charactist cours off guidanc decis make futur eavor unveil pathwai factor advs sourc systemat pubm april select obsv charactist advs consid elig scrutin ovlap extract synthesi factor point extract size crude elig size ovlap synthes appli main admiss icu mechan ventil mortal com elig uniqu factor seven strongest iolog evid statist valu hetogen sampl size intv exclud null valu absenc small elev protein decreas lymph

In [ ]:
print(target_df.title[0])
print(smoke_.title[6457])

Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019 (COVID-19)-Associated Hospitalization Surveillance Network (COVID-NET)
Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019 (COVID-19)-Associated Hospitalization Surveillance Network (COVID-NET)


**Apparently we have all abstracts in the original data, just some titles are not found somehow**



> Since the first article wasn't found, and the second one had duplicates, we begin with the third article:




In [ ]:
similars2 = d2v_model.docvecs.most_similar(positive=[d2v_target[0]], topn=2)
print(similars2)

[(6457, 0.9717298150062561), (9544, 0.8475455045700073)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print(target_df.title[0])
print(smoke_.title[9544])

Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019 (COVID-19)-Associated Hospitalization Surveillance Network (COVID-NET)
Early estimation of the risk factors for hospitalisation and mortality by COVID-19 in Mexico


In [ ]:
# Run the similarity test for all found articles:
# FOR ALL TARGET ARTICLES!
# All ABSTRACTS FOUND
"""similar_passed = []
similar_failed = []
for i in range(len(index_abstract_found)):
   target_i = index_abstract_found[i][0]
   sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[target_i]], topn=2)
   if sim_test[0][0] == index_abstract_found[i][1]:
     similar_passed.append([index_abstract_found[i][0], sim_test[0][0], sim_test[0][1], sim_test[1][0], sim_test[1][1]])
   else:
     similar_failed.append([index_abstract_found[i], sim_test])"""

### continue with found abstract similarsss

In [ ]:
# Run the similarity test for all found articles:
target_similar = []
for i in range(len(index_abstract_found)):
   target_i = index_abstract_found[i][0]
   sim_test = d2v_model.docvecs.most_similar(positive=[d2v_target[target_i]], topn=2)
   if sim_test[0][0] == index_abstract_found[i][1]:
     # preparing the list to include all info
     target_similar.append([target_i, target_df.title[target_i], smoke_target[target_i], 
                            sim_test[0][0], sim_test[0][1], smoke_.title[sim_test[0][0]], smoke_data[sim_test[0][0]],
                            sim_test[1][0], sim_test[1][1], smoke_.title[sim_test[1][0]], smoke_data[sim_test[1][0]]])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
target_similar[0]

[0,
 'Interim Analysis of Risk Factors for Severe Outcomes among a Cohort of Hospitalized Adults Identified through the U.S. Coronavirus Disease 2019 (COVID-19)-Associated Hospitalization Surveillance Network (COVID-NET)',
 'unit state grea death global factor outcom adult aliz design cohort studi identifi at aliz surveil network set al counti state aliz laboratori confirm march ement ag sex race ethnic underli medic condit nineti percent underli condit requir intens unit icu admiss invas mechan ventil vasopressor di aliz independ factor at icu admiss ag year versu year adjust ratio arr respect male sex arr obes arr immunosuppress arr diabet arr independ factor at mortal ag year versu year arr respect male sex arr immunosuppress arr renal arr chronic lung arr cardiovascular arr neurolog disord arr diabet arr race ethnic at icu admiss death limit limit discharg di complet chart abstract aliz access medic record exclud conclus mortal markedli ing ag help character person est at outcom de

In [ ]:
df_colum = ['Target_index', 'Target_title', 'Target_abstract', 
            'Most_sim_index', 'Most_sim_percnt', 'Most_sim_title', 'Most_sim_abstract', 
            '2_most_sim_index', '2_most_sim_percnt','2_most_sim_title', '2_most_sim_abstract']
target_similar_df = pd.DataFrame(target_similar, index=range(len(target_similar)), columns=df_colum)
target_similar_df

,Target_index,Target_title,Target_abstract,Most_sim_index,Most_sim_percnt,Most_sim_title,Most_sim_abstract,2_most_sim_index,2_most_sim_percnt,2_most_sim_title,2_most_sim_abstract
0,0,Interim Analysis of Risk Factors for Severe Ou...,unit state grea death global factor outcom adu...,3786,0.978154,Interim Analysis of Risk Factors for Severe Ou...,unit state grea death global factor outcom adu...,19596,0.887071,Low testosterone levels predict clinical adver...,new respiratori syndrom stress iv diagnost pro...
1,1,Predictors of mortality in hospitalized COVID-...,mortal continu rise world inform predictor mor...,710,0.985562,Predictors of mortality in hospitalized COVID-...,mortal continu rise world inform predictor mor...,1329,0.862497,Clinical and laboratory findings from patients...,decemb experienc novel contin aim consid chara...
2,2,Factors Associated With Intubation and Prolong...,identifi factor at intub extub aliz studi desi...,10829,0.992443,Factors Associated With Intubation and Prolong...,identifi factor at intub extub aliz studi desi...,15244,0.837286,Association of Obesity with Disease Severity a...,explor potenti ation obes chronic outcom inten...
3,3,Risk factors for adverse clinical outcomes in ...,alli heterogen vari iti prognosi characterist ...,3305,0.991325,Risk factors for adverse clinical outcomes in ...,alli heterogen vari iti prognosi characterist ...,11896,0.821423,The impact of 2019 novel coronavirus on heart ...,evid cardiac injuri inconsist aim summar avail...
4,4,Severe obesity is associated with higher in-ho...,new york epicent underrepres minor preval obes...,13033,0.987122,Severe obesity is associated with higher in-ho...,new york epicent underrepres minor preval obes...,6106,0.923005,Severe obesity is associated with higher in-ho...,new york epicent underrepres minor preval obes...
5,5,"Clinical Characteristics, Risk Factors and Out...",kuwait prior countri mass screen incom travel ...,3029,0.988544,"Clinical Characteristics, Risk Factors and Out...",kuwait prior countri mass screen incom travel ...,14414,0.792233,"COVID-19 Testing, Epidemic Features, Hospital ...",unit states’ grown extens februari substanti a...
6,6,Early estimation of the risk factors for hospi...,preval chronic non degen suspect mexico fatal ...,6335,0.975533,Early estimation of the risk factors for hospi...,preval chronic non degen suspect mexico fatal ...,4584,0.852167,Covid-19 by Race and Ethnicity: A National Coh...,grow concern racial ethnic minor commun world ...
7,7,Smoking Prevalence is Low in Symptomatic Patie...,littl inform role activ smoke predispos ion re...,4122,0.976501,Smoking Prevalence is Low in Symptomatic Patie...,littl inform role activ smoke predispos ion re...,7985,0.854642,Systematic review of the prevalence of current...,smoke corona unknown purpos studi systemat exa...
8,8,OpenSAFELY: factors associated with COVID-19-r...,establish novel rapidli aris death requir new ...,2665,0.994789,OpenSAFELY: factors associated with COVID-19-r...,establish novel rapidli aris death requir new ...,11050,0.803210,Estimating excess 1-year mortality associated ...,medic societ econom impact unknown overal popu...
9,9,30-day mortality in patients hospitalized with...,itali european countri hit est record death eu...,2207,0.991813,30-day mortality in patients hospitalized with...,itali european countri hit est record death eu...,9490,0.855972,Coronavirus Disease-2019 in Heart Transplant R...,identifi wuhan decemb risen exponenti characte...
